In [3]:
# Test NREL Battery Model
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# Try to import NREL PySAM battery model
try:
    import PySAM.Battery as battery
    NREL_AVAILABLE = True
    print("NREL PySAM Battery model available")
except ImportError:
    NREL_AVAILABLE = False
    print("NREL PySAM Battery model not available")

# Function to simulate battery with NREL model
def simulate_battery_nrel(
    pv_gen: pd.Series,
    demand: pd.Series,
    battery_kwh: float,
    battery_kw: float = None,
    roundtrip_eff: float = 0.9,
    min_soc_pct: float = 0.05  # 5% min SOC
):
    """
    Simulate battery dispatch using NREL's PySAM Battery model.
    """
    if not NREL_AVAILABLE:
        print("NREL PySAM Battery model not available. Install with 'pip install nrel-pysam'")
        return None, None
    
    if battery_kwh <= 0:
        print("Battery capacity must be > 0")
        return None, None
    
    # Set power rating if not provided
    if battery_kw is None:
        battery_kw = 0.5 * battery_kwh  # Default to 0.5C
    
    # Get time step in hours
    timestamps = pv_gen.index
    if len(timestamps) < 2:
        print("Need at least 2 timestamps to calculate time step")
        return None, None
    
    delta_h = (timestamps[1] - timestamps[0]).total_seconds() / 3600.0
    print(f"Time step: {delta_h} hours")
    
    # Convert Series to arrays
    pv_array = pv_gen.values
    demand_array = demand.values
    
    # Create a new standalone battery model
    batt = battery.default("StandaloneBatterySingleOwner")
    print("Created StandaloneBatterySingleOwner model")
    
    # Configure the battery - these parameters come from the documentation
    print("\nConfiguring battery parameters...")
    
    # 1. Simulation settings
    batt.Simulation.timestep_minutes = delta_h * 60  # Convert hours to minutes
    
    # 2. BatterySystem parameters
    batt.BatterySystem.batt_ac_or_dc = 1  # AC-connected
    batt.BatterySystem.batt_ac_dc_efficiency = roundtrip_eff * 100  # Convert to percentage
    batt.BatterySystem.batt_dc_ac_efficiency = roundtrip_eff * 100  # Convert to percentage
    batt.BatterySystem.batt_dc_dc_efficiency = 100  # Direct DC-DC is 100% efficient
    batt.BatterySystem.en_batt = 1  # Enable battery
    batt.BatterySystem.en_standalone_batt = 1  # Enable standalone mode
    
    # 3. BatteryCell parameters
    batt.BatteryCell.batt_chem = 1  # Li-ion
    batt.BatteryCell.batt_Qfull = 100  # Ah - will be scaled by model
    batt.BatteryCell.batt_minimum_SOC = min_soc_pct * 100  # Convert to percentage
    batt.BatteryCell.batt_maximum_SOC = 95  # Default max SOC %
    batt.BatteryCell.batt_initial_SOC = 50  # Start at 50% SOC
    
    # 4. Load parameters
    # Convert from array to list for PySAM
    batt.Load.load = demand_array.tolist()
    
    # 5. SystemOutput parameters
    # Set the generation from our PV array 
    batt.SystemOutput.gen = pv_array.tolist()
    
    # 6. BatteryDispatch parameters
    # We'll use automated dispatch mode
    batt.BatteryDispatch.batt_dispatch_choice = 0  # Automated dispatch
    batt.BatteryDispatch.batt_dispatch_auto_can_charge = 1  # Allow charging
    batt.BatteryDispatch.batt_dispatch_auto_can_gridcharge = 0  # No grid charging
    batt.BatteryDispatch.batt_dispatch_auto_can_clipcharge = 1  # Allow charging from clipped power
    
    # Execute the model
    print("\nExecuting battery model...")
    batt.execute()
    print("Battery model execution complete")
    
    # Process the results
    try:
        # Get the outputs from the model
        print("\nProcessing model results...")
        
        # Extract battery state of charge
        batt_soc = np.array(batt.Outputs.batt_SOC)
        print(f"SOC data points: {len(batt_soc)}, range: {min(batt_soc)}-{max(batt_soc)}%")
        
        # Extract battery power
        batt_power = np.array(batt.Outputs.batt_power)
        print(f"Power data points: {len(batt_power)}, range: {min(batt_power)}-{max(batt_power)} kW")
        
        # Additional metrics
        batt_cycles = np.array(batt.Outputs.batt_cycles)
        batt_capacity_percent = np.array(batt.Outputs.batt_capacity_percent)
        
        # Create a DataFrame with the results
        df = pd.DataFrame({
            'timestamp': list(range(len(batt_soc))),
            'pv_gen': pv_array,
            'demand': demand_array,
            'battery_soc': batt_soc,
            'battery_power': batt_power
        })
        
        # Calculate battery charge and discharge
        df['battery_charge'] = np.where(batt_power > 0, batt_power * delta_h, 0)
        df['battery_discharge'] = np.where(batt_power < 0, -batt_power * delta_h, 0)
        
        # Calculate key metrics
        total_charge = df['battery_charge'].sum()
        total_discharge = df['battery_discharge'].sum()
        cycles = total_charge / battery_kwh
        
        # Calculate final capacity
        if len(batt_capacity_percent) > 0:
            final_capacity_pct = batt_capacity_percent[-1]
        else:
            final_capacity_pct = 100
        
        # Calculate energy flows
        df['pv_direct'] = np.minimum(df['pv_gen'], df['demand'])
        df['pv_excess'] = np.maximum(0, df['pv_gen'] - df['demand'])
        df['grid_import'] = np.maximum(0, df['demand'] - df['pv_direct'] - df['battery_discharge'])
        df['grid_export'] = np.maximum(0, df['pv_excess'] - df['battery_charge'])
        
        # Summarize results
        totals = {
            'total_demand': df['demand'].sum(),
            'total_pv_generation': df['pv_gen'].sum(),
            'total_pv_direct': df['pv_direct'].sum(),
            'total_battery_charge': total_charge,
            'total_battery_discharge': total_discharge,
            'total_grid_import': df['grid_import'].sum(),
            'total_grid_export': df['grid_export'].sum(),
            'battery_cycles': cycles,
            'final_capacity_percent': final_capacity_pct,
            'roundtrip_efficiency': total_discharge / (total_charge if total_charge > 0 else 1) * 100
        }
        
        print("\nSimulation Summary:")
        for key, value in totals.items():
            print(f"  • {key}: {value:.2f}")
        
        return df, totals
        
    except Exception as e:
        print(f"Error processing results: {e}")
        # Print all available outputs for debugging
        print("\nAvailable outputs:")
        for output in dir(batt.Outputs):
            if not output.startswith('_'):
                print(f"  • {output}")
        return None, None

# Load data from PV_Generation_excel.csv
print("\nLoading data from PV_Generation_excel.csv...")

# Use relative paths to find the data file
possible_paths = [
    Path.cwd() / "data" / "PV_Generation_excel.csv",
    Path.cwd().parent / "data" / "PV_Generation_excel.csv",
    Path.cwd().parent.parent / "data" / "PV_Generation_excel.csv",
    Path("/Users/petertunali/Documents/GitHub/Battery_Optimisation/data/PV_Generation_excel.csv")
]

csv_path = None
for path in possible_paths:
    if path.exists():
        csv_path = path
        print(f"Found data file at: {path}")
        break

if csv_path is None:
    print("Could not find PV_Generation_excel.csv in any of the expected locations.")
else:
    # Load the CSV
    df = pd.read_csv(csv_path, parse_dates=["Date and Time"], dayfirst=True)
    
    # Use the specific column names we found
    pv_col = "PV Generated (kWh)"
    consumption_col = "Consumtpion (kWh)"  # Noticed the typo in your column name
    
    print(f"\nUsing columns:")
    print(f"  • PV generation: '{pv_col}'")
    print(f"  • Consumption: '{consumption_col}'")
    
    # Get the demand and PV generation series
    pv_gen = pd.Series(df[pv_col].values, index=df["Date and Time"])
    demand = pd.Series(df[consumption_col].values, index=df["Date and Time"])
    
    print(f"Loaded {len(pv_gen)} data points from {pv_gen.index.min()} to {pv_gen.index.max()}")
    print(f"Total PV generation: {pv_gen.sum():.2f} kWh")
    print(f"Total demand: {demand.sum():.2f} kWh")
    
    # Test the NREL battery model with a 10 kWh battery
    battery_kwh = 10.0
    battery_kw = 5.0
    
    print(f"\nTesting NREL battery model with {battery_kwh} kWh / {battery_kw} kW battery...")
    results_df, results_totals = simulate_battery_nrel(
        pv_gen, demand, battery_kwh, battery_kw
    )
    
    if results_df is not None:
        # Plot results for a few days
        plt.figure(figsize=(15, 10))
        
        # Plot 1: PV, demand, and battery state
        plt.subplot(2, 1, 1)
        days_to_plot = 3  # Show 3 days of data
        start_idx = 0
        end_idx = min(48 * days_to_plot, len(results_df))  # 48 half-hour periods per day
        
        plt.plot(results_df['timestamp'][start_idx:end_idx], results_df['pv_gen'][start_idx:end_idx], 'orange', label='PV Generation')
        plt.plot(results_df['timestamp'][start_idx:end_idx], results_df['demand'][start_idx:end_idx], 'blue', label='Demand')
        plt.plot(results_df['timestamp'][start_idx:end_idx], results_df['battery_power'][start_idx:end_idx], 'green', label='Battery Power (+ charge, - discharge)')
        
        # Plot SOC on secondary axis
        ax2 = plt.gca().twinx()
        ax2.plot(results_df['timestamp'][start_idx:end_idx], results_df['battery_soc'][start_idx:end_idx], 'red', label='Battery SOC (%)')
        ax2.set_ylabel('Battery SOC (%)')
        ax2.set_ylim(0, 100)
        
        plt.title(f'NREL Battery Model: First {days_to_plot} Days')
        plt.xlabel('Time Step')
        plt.ylabel('Power (kW)')
        plt.legend(loc='upper left')
        ax2.legend(loc='upper right')
        plt.grid(True)
        
        # Plot 2: Energy flows
        plt.subplot(2, 1, 2)
        plt.plot(results_df['timestamp'][start_idx:end_idx], results_df['pv_direct'][start_idx:end_idx], 'green', label='PV Used Directly')
        plt.plot(results_df['timestamp'][start_idx:end_idx], results_df['battery_charge'][start_idx:end_idx], 'blue', label='Battery Charging')
        plt.plot(results_df['timestamp'][start_idx:end_idx], results_df['battery_discharge'][start_idx:end_idx], 'red', label='Battery Discharging')
        plt.plot(results_df['timestamp'][start_idx:end_idx], results_df['grid_import'][start_idx:end_idx], 'black', label='Grid Import')
        plt.plot(results_df['timestamp'][start_idx:end_idx], results_df['grid_export'][start_idx:end_idx], 'orange', label='PV Export')
        
        plt.title('Energy Flows')
        plt.xlabel('Time Step')
        plt.ylabel('Energy (kWh)')
        plt.legend()
        plt.grid(True)
        
        plt.tight_layout()
        plt.show()

NREL PySAM Battery model available

Loading data from PV_Generation_excel.csv...
Found data file at: /Users/petertunali/Documents/GitHub/Battery_Optimisation/data/PV_Generation_excel.csv

Using columns:
  • PV generation: 'PV Generated (kWh)'
  • Consumption: 'Consumtpion (kWh)'
Loaded 17570 data points from 2025-01-01 00:00:00 to 2025-12-31 23:30:00
Total PV generation: 6663.23 kWh
Total demand: 43953.65 kWh

Testing NREL battery model with 10.0 kWh / 5.0 kW battery...
Time step: 0.5 hours
Created StandaloneBatterySingleOwner model

Configuring battery parameters...

Executing battery model...


Exception: battery execution error.
	exec fail(battery): Electric load must have either the same time step as the simulation timestep, or 8760 time steps.

